In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import pickle

import numpy as np

import chemistry
from pprint import pprint

In [7]:
with open('../data/molecules.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [11]:
from chemistry import Molecule, atomic_number

def enhance_molecule(m):
    mnew = Molecule.copy(m)
    
    atomic_number = np.array([chemistry.atomic_number[a] for a in m.symbols], dtype='float32')
    mnew.simple_field_intensity = np.empty(m.n_atoms, dtype='float32')
    mnew.field_intensity = np.empty(m.n_atoms, dtype='float32')
    
    mnew.force_per_pair = np.empty((m.n_atoms, m.n_atoms), dtype='float32')
    mnew.force = np.empty(m.n_atoms, dtype='float32')

    for i in range(m.n_atoms):
        pos_this = np.tile(m.positions[i], (m.n_atoms, 1))
        pos_other = m.positions
        diff = pos_this - pos_other
        dist2 = np.sum(diff * diff, axis=1)
        
        dist = np.sqrt(dist2)
        dist = np.tile(dist.reshape((m.n_atoms, 1)), (1, 3))
        dist[i, :] = 1
        
        direc = diff / dist
        
        fi_per_atom = atomic_number.copy()
        sel = np.arange(m.n_atoms) != i
        fi_per_atom[sel] /= dist2[sel]
        fi_per_atom[i] = 0
        
        fi_tile = np.tile(fi_per_atom.reshape((m.n_atoms, 1)), (1, 3))
        fi_vec = fi_tile * direc
        
        force = atomic_number.copy()
        force[sel] *= atomic_number[i]
        force[sel] /= dist2[sel]
        force[i] = 0
        
        force_tile = np.tile(force.reshape((m.n_atoms, 1)), (1, 3))
        force_vec = force_tile * direc
        
        mnew.simple_field_intensity[i] = fi_per_atom.sum()
        mnew.field_intensity[i] = np.linalg.norm(fi_vec.sum(axis=0))
        
        mnew.force_per_pair[i, :] = force[:]
        mnew.force[i] = np.linalg.norm(force_vec.sum(axis=0))
        

    return mnew

    
enhance_molecule(molecules['dsgdb9nsd_000001'])

Name: dsgdb9nsd_000001
Atoms:
  C 0: [-0.01269814  1.0858041   0.008001  ]
  H 1: [ 0.00215042 -0.00603132  0.00197612]
  H 2: [1.0117308e+00 1.4637512e+00 2.7657481e-04]
  H 3: [-0.54081506  1.4475266  -0.8766437 ]
  H 4: [-0.5238136  1.4379326  0.9063973]
Bonds:
  C(0) - H(1)
  C(0) - H(2)
  C(0) - H(3)
  C(0) - H(4)
Simple field intensity :
  C(0) - 3.354708433151245
  H(1) - 5.975549221038818
  H(2) - 5.975560188293457
  H(3) - 5.975599765777588
  H(4) - 5.975588798522949
Field intensity :
  C(0) - 3.05920657410752e-05
  H(1) - 5.802408695220947
  H(2) - 5.802420616149902
  H(3) - 5.802464962005615
  H(4) - 5.802454471588135


In [12]:
molecules_enh = {mn:enhance_molecule(molecules[mn]) for mn in molecules}

In [13]:
with open('../data/molecules_enh.pickle', 'wb') as f:
    pickle.dump(molecules_enh, f)